# Emission Model Configuration
In this notebook, we'll configure the emission model to be used by our AR simulations. This will include many unresolved lines as well as a few resolved lines that will be observed by *Hinode*/EIS. The information regarding the resolved lines will be taken from Warren et al. (2012).

In [1]:
import os
import urllib
import io
import logging
logging.basicConfig(level=logging.INFO)

import pandas
import h5py
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u
import astropy.constants as const
import ChiantiPy.core as ch
import ChiantiPy.tools.util as ch_tools_util
import ChiantiPy.tools.data as ch_data

from synthesizAR.atomic import EmissionModel,ChIon

%matplotlib inline

 using cli
 using CLI for selections
 reading chiantirc file


/home/wtb2/anaconda3/envs/systematic-ar-study/lib/python3.5/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)
/home/wtb2/anaconda3/envs/systematic-ar-study/lib/python3.5/site-packages/matplotlib/__init__.py:1401: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


## Read in Data from Warren et al. (2012)

In [ ]:
tmp = (urllib.request.urlopen('http://iopscience.iop.org/0004-637X/759/2/141/suppdata/apj446760t2_mrt.txt')
                     .readlines())

In [ ]:
col_names = [str(t.strip()).split('  ')[-1] for t in tmp[8:15]]
col_names = [c[:-1] for c in col_names[0:2]] + ['Element','Ion','Wavelength'] + [c[:-1] for c in col_names[3:]]

In [ ]:
table_io = io.StringIO(','.join(col_names) + '\n' 
                       + '\n'.join([','.join(filter(None,t.strip().decode('utf8').split(' '))) for t in tmp[19:]]))

In [ ]:
df = pandas.read_csv(table_io)

In [ ]:
df

## Selecting Ions
Select the number of (unresolved) ions in addition to the resolved lines for the active region. For AR 9 and 10, we'll create distinct emission models based on the lines selected by Warren et al. (2012).

We need to correct each of the resolved wavelengths to make sure that they can be found in the CHIANTI database.

In [ ]:
df_1109 = df[df['Active region']==9]

In [ ]:
resolved_lines_1109 = {}
for i,r in df_1109[df_1109['Instrument (1)']=='EIS'].iterrows():
    # parse ion name
    ion_name = ch_tools_util.spectroscopic2name(r['Element'],r['Ion'])
    print('Processing ion {}'.format(ion_name))
    # correct wavelength
    w = float(r['Wavelength'])
    tmp = ch.ion(ion_name)
    new_w = min(tmp.Wgfa['wvl'],key=lambda x:abs(x-w))
    if abs(new_w - w)>0.1:
        raise ValueError('Closest wavelength {} is not within tolerance of {}'.format(new_wvl,w))
    w = new_w
    # save
    if ion_name not in resolved_lines_1109:
        resolved_lines_1109[ion_name] = [w]
    else:
        resolved_lines_1109[ion_name].append(w)

In [ ]:
all_ion_list_1109 = []
for ion in ch_data.MasterList:
    if ion[-1]=='d':
        continue
    tmp = {'name':ion,
           'resolved_wavelengths':[]
          }
    if ion in resolved_lines_1109:
        tmp['resolved_wavelengths'] = resolved_lines_1109[ion]
    tmp['resolved_wavelengths'] = tmp['resolved_wavelengths']*u.angstrom
    all_ion_list_1109.append(tmp)

## Create Emission Model
Now, pass the complete ion list to the emission model. In this way, we've told the emission model to only calculate the wavelength-resolved emission from those ions we picked from the Warren et al. list. We're calculating the 

In [ ]:
emission_model_1109 = EmissionModel(all_ion_list_1109,
                                    temperature=np.logspace(5,8,50)*u.K,
                                    density=np.logspace(8,11,20)/(u.cm**3),
                                    energy_unit='photon',
                                    chianti_db_filename='/data/datadrive1/ar_forward_modeling/systematic_ar_study/chianti_db.h5'
                                   )

Restore the emissivity model.

In [2]:
emission_model_1109 = EmissionModel.restore('/data/datadrive1/ar_forward_modeling/systematic_ar_study/emission_model1109')

INFO:EmissionModel:Using CHIANTI HDF5 database in /data/datadrive1/ar_forward_modeling/systematic_ar_study/chianti_db.h5
INFO:EmissionModel:Creating ion h_1
INFO:EmissionModel:Creating ion he_2
INFO:EmissionModel:Creating ion c_6
INFO:EmissionModel:Creating ion n_7
INFO:EmissionModel:Creating ion o_8
INFO:EmissionModel:Creating ion ne_10
INFO:EmissionModel:Creating ion na_11
INFO:EmissionModel:Creating ion mg_12
INFO:EmissionModel:Creating ion al_13
INFO:EmissionModel:Creating ion si_14
INFO:EmissionModel:Creating ion p_15
INFO:EmissionModel:Creating ion s_16
INFO:EmissionModel:Creating ion cl_17
INFO:EmissionModel:Creating ion ar_18
INFO:EmissionModel:Creating ion k_19
INFO:EmissionModel:Creating ion ca_20
INFO:EmissionModel:Creating ion fe_26
INFO:EmissionModel:Creating ion ni_28
INFO:EmissionModel:Creating ion he_1
INFO:EmissionModel:Creating ion c_5
INFO:EmissionModel:Creating ion n_6
INFO:EmissionModel:Creating ion o_7
INFO:EmissionModel:Creating ion ne_9
INFO:EmissionModel:Creati

In [3]:
emission_model_1109.calculate_emissivity()

INFO:EmissionModel:Calculating emissivity for ion h_1
INFO:ChIon:Expressing emissivity in units of photons
INFO:ChIon:Calculating level populations.
INFO:ChIon:Calculating emissivity
INFO:EmissionModel:Calculating emissivity for ion he_2
INFO:ChIon:Expressing emissivity in units of photons
INFO:ChIon:Calculating level populations.
INFO:ChIon:Calculating emissivity
INFO:EmissionModel:Calculating emissivity for ion c_6
INFO:ChIon:Expressing emissivity in units of photons
INFO:ChIon:Calculating level populations.
INFO:ChIon:Calculating emissivity
INFO:EmissionModel:Calculating emissivity for ion n_7
INFO:ChIon:Expressing emissivity in units of photons
INFO:ChIon:Calculating level populations.
INFO:ChIon:Calculating emissivity
INFO:EmissionModel:Calculating emissivity for ion o_8
INFO:ChIon:Expressing emissivity in units of photons
INFO:ChIon:Calculating level populations.
INFO:ChIon:Calculating emissivity
INFO:EmissionModel:Calculating emissivity for ion ne_10
INFO:ChIon:Expressing emissi

Save the emissivity model.

In [4]:
emission_model_1109.save(savedir='/data/datadrive1/ar_forward_modeling/systematic_ar_study/emission_model1109')

INFO:EmissionModel:Saving emission model information in /data/datadrive1/ar_forward_modeling/systematic_ar_study/emission_model1109
